# Import

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace
from transformers import get_linear_schedule_with_warmup

# Hyperparameter

In [15]:
config = {
    "learning_rate": 2e-5,
    "epoch": 8,
    "batch_size": 64
}

CFG = SimpleNamespace(**config)

# Load Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GBT해커톤/Data/train_df_1012.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GBT해커톤/Data/test_df_1012.csv")

In [5]:
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54314 entries, 0 to 54313
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54314 non-null  object
 1   분류      54314 non-null  object
 2   제목      54314 non-null  object
 3   키워드     54314 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   제목      23405 non-null  object
 2   키워드     23405 non-null  object
dtypes: object(3)
memory usage: 548.7+ KB


# Load Model

In [6]:
# # 토크나이저와 모델 변경
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from transformers import BertTokenizer, RobertaForSequenceClassification

In [7]:
CFG = SimpleNamespace(**config)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base", num_labels=len(train_df['분류'].unique())).to(device)
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
if torch.cuda.is_available():
    print(True)
else:
    print(False)

True


# Custom Dataset

In [20]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64): # 클래스 초기화 (__init__ 메서드)
        self.texts = texts                                      # texts: 텍스트 데이터의 리스트입니다.
        self.labels = labels                                    # labels: 각 텍스트에 대한 레이블(정답)의 리스트입니다. 레이블이 없는 경우를 위해 기본값을 None으로 설정할 수 있습니다.
        self.tokenizer = tokenizer                              # tokenizer: 텍스트를 토큰화하는 데 사용할 토크나이저 객체입니다. 주로 BERT와 같은 모델의 토크나이저를 사용합니다.
        self.max_len = max_len                                  # max_len: 토큰화된 텍스트의 최대 길이입니다. 기본값은 128로 설정되어 있습니다.

    def __len__(self):                                          # 데이터셋 길이 (__len__ 메서드)
        return len(self.texts)                                  # 데이터셋의 크기를 반환합니다. 즉, 텍스트의 개수를 반환합니다.

    def __getitem__(self, item):                                # 데이터 항목 가져오기 (__getitem__ 메서드)
        text = str(self.texts[item])                            # item: 데이터셋에서 가져올 특정 인덱스입니다.
        label = self.labels[item] if self.labels is not None else -1 # label: 해당 인덱스의 레이블을 가져옵니다. 레이블이 없으면 -1로 설정합니다.
        encoding = self.tokenizer.encode_plus(                  # encoding: 토크나이저의 encode_plus 메서드를 사용하여 텍스트를 인코딩합니다. 이 과정에서 다음과 같은 옵션을 설정합니다:
            text,                                              #
            add_special_tokens=True,                            # add_special_tokens=True: 모델에 필요한 특별한 토큰을 추가합니다.
            max_length=self.max_len,                            # max_length=self.max_len: 최대 길이를 설정합니다.
            return_token_type_ids=False,                        #
            padding='max_length',                               # padding='max_length': 최대 길이에 맞춰 패딩을 추가합니다.
            truncation=True,                                    # truncation=True: 최대 길이를 초과하는 경우 잘라냅니다.
            return_attention_mask=True,                         # return_attention_mask=True: 어텐션 마스크도 반환합니다.
            return_tensors='pt',                                # return_tensors='pt': PyTorch 텐서 형식으로 반환합니다.
        )
        return {                                                # 반환하는 딕셔너리에는 원본 텍스트, 인코딩된 입력 ID, 어텐션 마스크, 레이블이 포함
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


# Data Preprocessing

In [21]:
# 데이터 준비
# train_df['키워드'] = train_df['제목'] + ' ' + train_df['키워드']
# test_df['키워드'] = test_df['제목'] + ' ' + test_df['키워드']

# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.키워드.tolist(), train_df.label.tolist(), tokenizer)   # train_df에서 키워드와 label을 리스트로 변환하여 TextDataset 클래스의 인스턴스를 생성
val_dataset = TextDataset(val_df.키워드.tolist(), val_df.label.tolist(), tokenizer)         # val_df에서 키워드와 label을 사용하여 검증 데이터셋을 생성
test_dataset = TextDataset(test_df.키워드.tolist(), None, tokenizer)                        # test_df에서 키워드를 리스트로 변환하여 TextDataset을 생성합니다. 이 데이터셋은 레이블이 없으므로 None을 전달

# 데이터 로더 생성, DataLoader는 데이터를 미니 배치 단위로 나누어서 제공해주는 역할
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [23]:
train_dataset[1784]

{'text': '소유주 개방 주차장 시설비 주차장 개방 건축물 소유주 차단기 CCTV 설치비 신청 아파트 종교시설 대형 마트 상가 주차장 건물 부설 사용자 시간대 주민들 사용 도심 주차난 도심 주차 해소 부설 주차장 개방 건축물 소유주 개방 유지 조건 주차선 정비 차단기 CCTV 시설비 설치비용 한도 최대 신청서 제출서류 구비 시청 교통 정책 방문 우편 접수 예산 소진 마감 신청 평가 결과 개별통지 주차난 주택가 상가 밀집 주차 면수 선정 방침 백군기 시설 활용 공유형 주차장 주차 해소 기여 주차공간 부족 공영주차장 추가 조성 도심 다각적 추진',
 'input_ids': tensor([    0, 19207,  5751,  7086,  3953,  2151,  7086,  5751,  9233, 19207,
          6202,  2015, 10030,  4198,  2151,  4373,  4155,  4786, 10171,  4477,
          6435,  6682,  7086,  4515, 18384, 15259, 18374,  3972,  2031,  3704,
          6246,  5447,  2336,  6246,  5447,  5343, 18384,  7086,  5751,  9233,
         19207,  5751,  4036,  4423,  5447,  2020,  5336,  6202,  2015, 10030,
          3953,  2151,  4198, 20128,  7932,  3980, 12511,  4962,  2112,  2397,
         13092,  4479,  4352,     2]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

위의 텐서는 자연어 처리에서 토큰화된 텍스트를 나타내며, 각 숫자는 특정 토큰에 대한 ID를 의미한다. 출력된 텐서는 모델 학습이나 예측에 사용한다.

In [24]:
# 옵티마이저 및 학습 파라미터 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * CFG.epoch)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [86]:
# # 학습
# model.train()
# for epoch in range(CFG.epoch):
#     for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

#     # Validation
#     model.eval()
#     val_predictions = []
#     val_true_labels = []
#     with torch.no_grad():
#         for batch in tqdm(val_loader, desc='Validating'):
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)
#             outputs = model(input_ids, attention_mask=attention_mask)
#             _, preds = torch.max(outputs.logits, dim=1)
#             val_predictions.extend(preds.cpu().tolist())
#             val_true_labels.extend(labels.cpu().tolist())

#     # 검증 결과 출력
#     val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
#     print(f"Validation F1 Score: {val_f1:.2f}")

In [25]:
import numpy as np

# 초기 설정
best_f1 = -np.inf  # 가장 좋은 F1 점수 초기화
patience = 3  # 개선이 없을 경우 멈추기 전 허용할 에포크 수
counter = 0  # 개선이 없었던 에포크 수 카운터

model.train()
for epoch in range(CFG.epoch):
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())

    # 검증 결과 출력
    val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    print(f"Validation F1 Score: {val_f1:.2f}")

    # F1 점수가 개선되었는지 확인
    if val_f1 > best_f1:
        best_f1 = val_f1
        counter = 0  # 개선이 있었으므로 카운터 초기화
    else:
        counter += 1  # 개선이 없으므로 카운터 증가

    # 개선이 없었던 에포크 수가 patience를 초과하면 학습 중단
    if counter >= patience:
        print("Early stopping triggered.")
        break

Validating: 100%|██████████| 136/136 [00:15<00:00,  8.92it/s]


Validation F1 Score: 0.59


Validating: 100%|██████████| 136/136 [00:15<00:00,  8.89it/s]


Validation F1 Score: 0.63


Validating: 100%|██████████| 136/136 [00:15<00:00,  9.06it/s]


...

Validation F1 Score: 0.69


Validating: 100%|██████████| 136/136 [00:15<00:00,  9.05it/s]


Validation F1 Score: 0.69


Validating: 100%|██████████| 136/136 [00:15<00:00,  8.81it/s]

Validation F1 Score: 0.69


# Inference

In [26]:
# 테스트 세트 추론
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

Testing: 100%|██████████| 366/366 [00:41<00:00,  8.89it/s]


# Submission

In [27]:
submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GBT해커톤/Data/sample_submission.csv")
submission["분류"] = decoded_predictions

submission.to_csv("/content/drive/MyDrive/Colab Notebooks/GBT해커톤/submission/1012_roberta_base(batch_64_max64)__.csv", encoding='UTF-8-sig', index=False)

In [28]:
submission['분류'].value_counts()

,count
분류,
지역,11921
경제:부동산,1482
사회:사건_사고,1138
경제:반도체,1003
사회:사회일반,582
사회:교육_시험,464
사회:의료_건강,399
정치:국회_정당,387
경제:취업_창업,357
